# Coordinate Rotation Digital Computer (CORDIC) with OpenLane

```
Copyright 2022 Google LLC.
SPDX-License-Identifier: Apache-2.0
```



## Agenda

---

In this notebook, you will learn what [CORDIC](https://en.wikipedia.org/wiki/CORDIC) is and how to implement a simple CORDIC design thru the [OpenLane](https://github.com/The-OpenROAD-Project/OpenLane/) GDS to RTL flow targeting the open source [SKY130 PDK](https://github.com/google/skywater-pdk/). Specifically the following things:


*   What is CORDIC, How does it work, and What's special about it?
*   How to run the simulation and how does the waveform look like?
*   How to run end-to-end implementation?
*   How does Layout look like?
*   What's the metrics of CORDIC?

Another main purpose of this notebook is to provide a good starting point if in the future you want to design something else, simply replace the verilog files and everything should be good to go.




## Introduction
---

### What is Coordinate Rotation Digital Computer (CORDIC)?

### What's Special about CORDIC Hardware?

## Simulation


### Testbench Structure

### Simulation Waveform

## Implementation


In [ ]:
#@title Install Dependencies {display-mode: "form"}
#@markdown - Click the ▷ button to setup the digital design environment based on [conda-eda](https://github.com/hdl/conda-eda).

openlane_version = 'latest' #@param {type:"string"}
open_pdks_version = 'latest' #@param {type:"string"}

if openlane_version == 'latest':
  openlane_version = ''
if open_pdks_version == 'latest':
  open_pdks_version = ''

import os
import pathlib
import sys

!curl -Ls https://micro.mamba.pm/api/micromamba/linux-64/latest | tar -xj bin/micromamba
conda_prefix_path = pathlib.Path('conda-env')
site_package_path = conda_prefix_path / 'lib/python3.7/site-packages'
sys.path.append(str(site_package_path.resolve()))
CONDA_PREFIX = str(conda_prefix_path.resolve())
PATH = os.environ['PATH']
LD_LIBRARY_PATH = os.environ.get('LD_LIBRARY_PATH', '')
%env CONDA_PREFIX={CONDA_PREFIX}
%env PATH={CONDA_PREFIX}/bin:{PATH}
%env LD_LIBRARY_PATH={CONDA_PREFIX}/lib:{LD_LIBRARY_PATH}
!bin/micromamba create --yes --prefix $CONDA_PREFIX
!echo 'python ==3.7*' >> {CONDA_PREFIX}/conda-meta/pinned
!CI=0 bin/micromamba install --quiet --yes --prefix $CONDA_PREFIX \
                     --channel litex-hub \
                     --channel main \
                     openlane={openlane_version} \
                     open_pdks.sky130a={open_pdks_version}
!CI=0 bin/micromamba install --quiet --yes --prefix $CONDA_PREFIX \
                     --channel conda-forge \
                     gdstk

### Write CORDIC Verilogs

Since designing CORDIC from scratch is not the main target of this notebook, we have provided the verilog files under "cordic/rtl", the brief explanation of each files is as below



*   `angle_adjust.v`: 
*   `cordic.include`: 
*   `cordic.v`: 
*   `cordic_config.v`: 
*   `cordic_constant.v`: 
*   `cordic_iteration.v`: 
*   `cordic_top.v`: 
*   `cordic_unit.v`: 
*   `scale.v`: 



### Config CORDIC

There are however some user-defined parameters that would affect the hardware cost, therefore we expose such configuration files to the users based on the preference.

The parameters are:
*   `NUM_ITER`:
*   `NUM_STAGE`:
*   `EN_SCALE`:

In [ ]:
#@markdown - Please fill in the parameters below and click the ▷ button to config your CORDIC

NUM_ITER = 12 #@param {type:"integer"}
NUM_STAGE = 4 #@param {type:"integer"}
EN_SCALE = 1 #@param {type:"integer"}

#@markdown - Not sure why but this cell and the next one was working


In [ ]:
%%writefile cordic/rtl/cordic_config.v

`ifndef __CORDIC_CONFIG__
`define __CORDIC_CONFIG__

   `define NUM_ITER  = 12
   `define NUM_STAGE = 4
   `define EN_SCALE  = 1

`endif

### Write OpenLane Flow Configuration

For more information, please refer to [Documentation](https://openlane.readthedocs.io/en/latest/reference/configuration.html).

In [ ]:
%%writefile config.json
{
    "DESIGN_NAME": "cordic_top",
    "VERILOG_FILES": "dir::cordic/rtl/*.v",
    "CLOCK_PERIOD": 10,
    "CLOCK_NET": "i_clk",
    "CLOCK_PORT": "i_clk",

    "FP_SIZING": "absolute",
    "DIE_AREA": "0 0 500 500"
}

### Run OpenLane Flow

[OpenLane](https://openlane.readthedocs.io/en/latest/) is an automated [RTL](https://en.wikipedia.org/wiki/Register-transfer_level) to [GDSII](https://en.wikipedia.org/wiki/GDSII) flow based on several components including [OpenROAD](https://theopenroadproject.org/), [Yosys](https://yosyshq.net/yosys/), [Magic](http://www.opencircuitdesign.com/magic/), [Netgen](http://opencircuitdesign.com/netgen/) and custom methodology scripts for design exploration and optimization targeting [open source PDKs](https://github.com/google/open-source-pdks).

![img](https://openlane.readthedocs.io/en/latest/_images/flow_v1.png)

In [ ]:
%env PDK=sky130A
!flow.tcl -design .

### Display layout

In [ ]:
import pathlib
import gdstk
import IPython.display

gdss = sorted(pathlib.Path('runs').glob('*/results/final/gds/*.gds'))
library = gdstk.read_gds(gdss[-1])
top_cells = library.top_level()
top_cells[0].write_svg('cordic.svg')
IPython.display.SVG('cordic.svg')

### Metrics

For more information, please refer to [Documentation](https://openlane.readthedocs.io/en/latest/reference/datapoint_definitions.html).


In [ ]:
import pandas as pd
import pathlib

pd.options.display.max_rows = None
final_summary_reports = sorted(pathlib.Path('runs').glob('*/reports/metrics.csv'))
df = pd.read_csv(final_summary_reports[-1])
df.transpose()